# Udwadia Kalaba Example
Let's look at the simple pendulum using the [Udwadia-Kalaba equation](https://en.wikipedia.org/wiki/Udwadia%E2%80%93Kalaba_equation)

In [1]:
using Pkg
Pkg.activate(".")
# Pkg.add("SymPy")
using SymPy
using LinearAlgebra

In [2]:
m,g,l= symbols("m,g,l", positive=true)
t = symbols("t")
x = SymFunction("x")(t)
y = SymFunction("y")(t)
;

In [3]:
M = diagm(0=>[m,m])

2×2 Array{Sym,2}:
 m  0
 0  m

In [4]:
Q = [0, -m*g]

2-element Array{Sym,1}:
    0
 -g*m

In [5]:
A = [x y]

1×2 Array{Sym,2}:
 x(t)  y(t)

In [6]:
b  = - diff(x,t)^2 - diff(y,t)^2

            2             2
  /d       \    /d       \ 
- |--(x(t))|  - |--(y(t))| 
  \dt      /    \dt      / 

In [7]:
M12 = diagm(0=>[sqrt(m),sqrt(m)])

2×2 Array{Sym,2}:
 sqrt(m)        0
       0  sqrt(m)

In [8]:
M12*M12

2×2 Array{Sym,2}:
 m  0
 0  m

In [9]:
A*M12

1×2 Array{Sym,2}:
 sqrt(m)*x(t)  sqrt(m)*y(t)

In [10]:
P = A/M12

1×2 Adjoint{Sym,Array{Sym,1}}:
 x(t)/sqrt(m)  y(t)/sqrt(m)

In [11]:
Ppinv = transpose(P)*inv( P*transpose(P) )

2×1 Array{Sym,2}:
 x(t)/(sqrt(m)*(x(t)^2/m + y(t)^2/m))
 y(t)/(sqrt(m)*(x(t)^2/m + y(t)^2/m))

In [12]:
qdd = M\( Q + M12*Ppinv*(b - (A/M)*Q ) )

2×1 Array{Sym,2}:
        (g*y(t) - Derivative(x(t), t)^2 - Derivative(y(t), t)^2)*x(t)/(m*(x(t)^2/m + y(t)^2/m))
 (-g*m + (g*y(t) - Derivative(x(t), t)^2 - Derivative(y(t), t)^2)*y(t)/(x(t)^2/m + y(t)^2/m))/m

In [13]:
qdd = [simplify(q) for q in qdd]

2×1 Array{Sym,2}:
         (g*y(t) - Derivative(x(t), t)^2 - Derivative(y(t), t)^2)*x(t)/(x(t)^2 + y(t)^2)
 -(g*x(t)^2 + y(t)*Derivative(x(t), t)^2 + y(t)*Derivative(y(t), t)^2)/(x(t)^2 + y(t)^2)

Let's define some states
$$ z = [x, y, \dot x, \dot y]$$

In [14]:
z1, z2, z3, z4 = symbols("z1, z2, z3, z4")
rule1 = Dict( diff(x,t)=> z3, diff(y,t)=>z4 )
rule2 = Dict( x=>z1, y=>z2 )

Dict{Sym,Sym} with 2 entries:
  x(t) => z1
  y(t) => z2

In [15]:
qdd |> subs(rule1) |> subs(rule2)

2×1 Array{Sym,2}:
       z1*(g*z2 - z3^2 - z4^2)/(z1^2 + z2^2)
 -(g*z1^2 + z2*z3^2 + z2*z4^2)/(z1^2 + z2^2)

## What about getting A and b numerically?
We can use the (somewhat magical) [ForwardDiff](http://www.juliadiff.org/ForwardDiff.jl/stable/index.html) package to automatically differentiate $h$ top build $A$ and $b$.

In [16]:
# Pkg.add("ForwardDiff")
using ForwardDiff

In [17]:
h( z, l) = z[1]^2 + z[2]^2 - l^2

h (generic function with 1 method)

Let's make an example differentiation:

In [18]:
ForwardDiff.gradient( z-> h(z,1.) ,[1,1.] )

2-element Array{Float64,1}:
 2.0
 2.0

and I can also build the Hessian

In [19]:
ForwardDiff.hessian( z -> h(z,1.), [1, 2.] )

2×2 Array{Float64,2}:
 2.0  0.0
 0.0  2.0

Now I can build $A$

In [20]:
An(q,l) = transpose( ForwardDiff.gradient( x-> h(x,l), q) )

An (generic function with 1 method)

In [21]:
An([1.,1],0)

1×2 Transpose{Float64,Array{Float64,1}}:
 2.0  2.0

And $b$ (as derived in the OnDynamics notes on blackboard)

In [22]:
bn(q,qd,l) = -transpose(qd)*ForwardDiff.hessian( x -> h(x,l), q)*qd

bn (generic function with 1 method)

In [23]:
bn([1,1], [2,2], 1)

-16